In [14]:
import pandas as pd
import numpy as np
import cellstates as cs
from cellstates.chelpers import marker_scores
import scipy.io as sio
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import pickle as pkl

In [15]:
path='/scicore/home/doetsch/GROUP/scigrp/vargasSingleCell3_2males_added/'
pathold='/scicore/home/doetsch/GROUP/scigrp/vargasSingleCell3/'

In [16]:
pklfile=open("varagasSingleCell3_2males_added.pkl",'rb')
df=pkl.load(pklfile)
clusters=pkl.load(pklfile)
hierarchy_df=pkl.load(pklfile)
score_df=pkl.load(pklfile)
annotation=pkl.load(pklfile)
n_scale=pkl.load(pklfile)
lmbd=pkl.load(pklfile)
pklfile.close()
data = df.to_numpy().astype(int)

In [17]:
clst = cs.Cluster(data, lmbd, clusters, max_clusters=max(clusters)+1, num_threads=12, n_cache=1000)
#colors=["#a62728","#1f77b4","#17bec7","#7e2e99","#e377c2"]
colordict={"Tom":"#1f77b4","Adam":"#17bec7","Viole":"#e377c2","Ana":"#d62728","Eve":"#a62728","Fiona":"#d41f7d","John":"#4287f5","Melvin":"#03255c"}
colors = list(map(colordict.get, np.unique(annotation)))
print(np.unique(annotation))
cl, clsizes = np.unique(clusters, return_counts=True)

['Adam' 'Ana' 'Eve' 'Fiona' 'John' 'Melvin' 'Tom' 'Viole']


In [18]:
with open("/scicore/home/doetsch/GROUP/scigrp/vargasSingleCell3/utils.py") as f:
    exec(f.read())

In [19]:
(promidtable,genetable)=loadGeneNames(path+"OUTPUT/concat_mat/prom_expr_promoters_w_genes.tsv")
(promidtable,genetable)=loadGeneNames(path+"OUTPUT/concat_mat/prom_expr_promoters_w_genes.tsv",promidtable,genetable)

promids = score_df.columns.values
geneids = []
for promid in promids[3:]:
    added=False
    for gene in genetable:
        if promid in genetable[gene]:
            geneids.append(gene)
            added=True
            break
    if not added:
        geneids.append(promid)

In [ ]:
import os
os.environ['QT_QPA_PLATFORM']='offscreen'
cleangenelist=[]
typ="blank"
with open(path+"nb/genelist.txt", 'r') as genelist:
    first=True
    for g in genelist:
        if g[0]==">":
            if not first:
                makeBallPlot(96, cleangenelist,typ+"_96.pdf",n_scale=n_scale,plotpath=path+"nb/plots/",addExpression=True)
                cleangenelist=[]
            typ=g.strip().split()[0]
            print(typ)
            first=False
        elif len(g.strip().split())==2:
            gene=g.strip().split()[0]
            cleangenelist.append(gene)
        else:
            gene=g.strip()
            cleangenelist.append(gene)
    makeBallPlot(96, cleangenelist,typ+"_96.pdf",n_scale=n_scale,plotpath=path+"nb/plots/",addExpression=True)

>Epithelial
>Lipid
>Myoepithelial
Cytor not found
>Dark


In [ ]:
nc = 96 # number of clusters
merged_clusters = cs.clusters_from_hierarchy(hierarchy_df, cluster_init=clusters, steps= - nc + 1)
newick_string = cs.hierarchy_to_newick(hierarchy_df[-nc+1:], merged_clusters, cell_leaves=False)
t = Tree(newick_string, format=1)
ts = get_TreeStyle_attributes(t, merged_clusters, annotation, colors=colors,leaf_scale=0.05,normalize=True,showInternalNodeNames=True)
new_leaf_names = ["merged"+str(nc)+"C"+str(i) for i in np.arange(nc)]
name_dict = dict(zip(t.iter_leaf_names(),  new_leaf_names))
for key in name_dict:
    name_dict[key]=name_dict[key]+"_"+key
for l in t.iter_leaves():
    l.add_face(TextFace(name_dict[l.name],fsize=60), column=2)
# ts.show_leaf_name = True
t.render(path+'nb/plots/plotHierarchy96.pdf', tree_style=ts)

In [13]:
promidtable=loadGeneNames(path+"OUTPUT/concat_mat/prom_expr_promoters_w_genes.tsv")
promidtable=loadGeneNames(pathold+"OUTPUT/concat_mat/prom_expr_promoters_w_genes.tsv",promidtable)
dumpEmpressionMatrix(path+"nb/expressionMatrix_96_allgenes.csv",96,hierarchy_df,promidtable)
dumpEmpressionMatrix(path+"nb/expressionMatrix_48_allgenes.csv",48,hierarchy_df,promidtable)
dumpEmpressionMatrix(path+"nb/expressionMatrix_24_allgenes.csv",24,hierarchy_df,promidtable)
dumpEmpressionMatrix(path+"nb/expressionMatrix_full_allgenes.csv",len(clsizes),hierarchy_df,promidtable)